# 获取城市名单

In [79]:
import requests
import pyquery
import re
import pandas as pd
import random

In [80]:
user_agent = 'Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)'
url = 'http://data.acmr.com.cn/member/city/city_md.asp'
headers = {'User-Agent':user_agent}
r1 = requests.get(url = url,headers = headers)

In [81]:
r1.encoding = 'gbk'

In [82]:
pq = pyquery.PyQuery(r1.text)

In [83]:
maintext_tag = pq('body > font > table > tr > td > table > tr > td > table > tr > td > table > tr > td > table > tr > td > table.maintext') 

In [84]:
city_super = maintext_tag.eq(0).text()

In [85]:
city_super = city_super.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [86]:
city_metro = maintext_tag.eq(1).text()

In [87]:
city_metro = city_metro.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [88]:
len(city_metro)

22

In [89]:
city_big = maintext_tag.eq(2).text()

In [90]:
city_big = city_big.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [91]:
'内蒙','广西','宁夏'

('内蒙', '广西', '宁夏')

In [92]:
city_middle = maintext_tag.eq(3).text()
city_middle = city_middle.strip(r'[TOP]').split(':')[1].split(' ')[1:-1]

In [93]:
city_small = maintext_tag.eq(4).text()

In [94]:
city_small = city_small.strip(r'[TOP]').split(':')

In [95]:
city_small_list = city_small[1].strip(r'县级市').split(' ')[1:-1]
city_small_list.extend(city_small[2].split(' ')[1:-1])

In [96]:
population = ['超大','特大','大','中等','小']
city_list = [city_super,city_metro,city_big,city_middle,city_small_list]
with open('city_info.csv',mode = 'w',encoding = 'utf-8') as city_info:
    city_info.write('city,population\n')
    
    for i in range(len(city_list)):
        cities = city_list[i]
        for city in cities:
            city_info.write('{},{}\n'.format(city,population[i]))

# 数据清洗

In [97]:
city_df = pd.read_csv('city_info.csv')

In [98]:
city_df['province'] = city_df.city.str.extract(r'(.+?)省',expand=False)

province_other = ['北京市','天津市','上海市','重庆市','内蒙','新疆','广西','宁夏','西藏']
for i in province_other:
    city_df.loc[city_df.city.str.contains(i),'province'] = i

In [99]:
city_df['name'] = city_df['city'].str.extract(r'(.+?)市',expand=False)
city_df['Name'] = city_df['name']
city_df.loc[city_df['Name'].str.contains('省'),'name'] = city_df['name'].str.extract(r'省(.+)',expand=False)   
for i in province_other:
    city_df.loc[city_df.name.str.contains(i),'name'] = city_df.name.str.replace(i,'') 


city_df = city_df[['city','population','province','name']]

In [100]:
city_df['name_first'] = city_df.name.str.extract(r'(^.{1})',expand=False)
city_df['name_last'] = city_df.name.str.extract(r'(.{1}$)',expand=False)

In [101]:
def chinese_to_py(a):
    a = str(a.encode('unicode_escape'))[-5:-1].upper()
    b = dic_py[a]
    return b            

In [102]:
dic_py = {}
with open('unicode_py.txt','r') as f:
    for i in f.readlines():
        dic_py[i.split()[0]] = i.split()[1]

In [103]:
name_first_py_list = []
for a in city_df['name_first']:
    name_first_py_list.append(chinese_to_py(a))
    
city_df['name_first_py'] = pd.Series(name_first_py_list)

In [104]:
name_last_py_list = []
for a in city_df['name_last']:
    name_last_py_list.append(chinese_to_py(a))
    
city_df['name_last_py'] = pd.Series(name_last_py_list)

# 游戏

In [105]:
def get_basic(name_city):
    return city_df.loc[city_df['name'] == name_city,['population','province','name']] 

In [106]:
def get_info(name_city):
    url_city = 'https://baike.baidu.com/item/{}'.format(name_city)
    r2 = requests.get(url = url_city,headers = headers)
    r2.encoding = 'utf-8'
    pq2 = pyquery.PyQuery(r2.text)
    return name_city,pq2('body > div > div > div > div > div > div').text().split(name_city,1)[1] 

In [108]:
name_city = input('请输入开始城市：')
def game_continue(name_city):
    name_last_py = city_df.loc[city_df['name'] == name_city,'name_last_py']
    name_last_py = name_last_py.values[0][:-1]

    name_last = city_df.loc[city_df['name_first_py'].str.contains(name_last_py),'name'].values
    print(name_last)
    if name_last.any(0):
        name_city = random.choice(list(name_last))
        print('接龙城市:{}'.format(name_city))
        user_input = input('请按以下提示继续:\nc：继续游戏\nt：停止游戏\nb：城市基本信息\ni：详细信息\n')    
        if user_input == 'c':
            name_city = name_city
            game_continue(name_city)
        elif user_input == 't':
            print('游戏停止')
            return None
        elif user_input == 'b':
            print(get_basic(name_city))
            game_continue(name_city)
        elif user_input == 'i':
            print(get_info(name_city))
            game_continue(name_city)
        
        else:
            print('输入不正确，游戏结束')
            return None
    else:
        print('无可接龙的城市，游戏结束')
        return None

game_continue(name_city)

请输入开始城市：北京
['荆州' '景德镇' '荆门' '靖江' '井冈山' '景洪']
接龙城市:荆州
请按以下提示继续:
c：继续游戏
t：停止游戏
b：城市基本信息
i：详细信息
i
('荆州', '，古时又称“ 江陵 ”， 湖北省 地级市，位于湖北中南部、 长江 中游、 江汉平原 腹地，全市国土面积1.41万平方公里，总人口达690万（2017年），下辖荆州区、沙市区、江陵县、松滋市、公安县、石首市、监利县、洪湖市8个县市区和荆州开发区、华中农高区、纪南文旅区3个功能区。荆州是春秋战国时 楚国 都城所在地，是国务院公布的全国首批24座 国家历史文化名城 之一、 中国优秀旅游城市 、 国家园林城市 、重要的公路 交通枢纽 和长江港口城市。 [1] 2017年6月，荆州市被命名国家卫生城市。 [2] 荆州历史厚重、文化灿烂，是一座古老文化与现代文明交相辉映的滨江城市。“禹划九州，始有荆州。”荆州建城历史长达3000多年。自公元前689年楚国建都 纪南城 ，先后有6个朝代、34位帝王在此建都，是当之无愧的“ 帝王之都 ”。从“天下第一循吏” 孙叔敖 到明朝万历首辅 张居正 ，从荆州走出去的宰相达138位，是名副其实的“宰相之城”。从爱国主义诗人 屈原 到 李白 、 杜甫 ，大批文人墨客在荆州吟诗作赋，也是实至名归的“诗词之市”。 [1] 荆州科教资源丰富，是湖北第二大人才密集地，拥有科研院所300多家，高等院校8所，院士专家工作站50个。2016年，荆州全市 地区生产总值 （GDP）为1726 . 8亿元、增长7.3%，总量居全省第四位。 [1] 目录 1 历史沿革 ▪ 古代 ▪ 近代 2 行政区划 3 地理环境 ▪ 位置 ▪ 地形 ▪ 气候 4 自然资源 ▪ 矿产资源 ▪ 水资源 ▪ 土地资源 ▪ 生物资源 5 人口民族 ▪ 人口 ▪ 民族 6 政治体制 7 经济概况 ▪ 综合 ▪ 第一产业 ▪ 第二产业 ▪ 第三产业 8 社会事业 ▪ 文化 ▪ 城建 ▪ 教育 ▪ 卫生 ▪ 科技 ▪ 社保 9 历史文化 ▪ 精神 ▪ 民俗 ▪ 美食 ▪ 方言 10 旅游名胜 11 著名人物 12 城市荣誉 13 友好城市 文化名城——荆州 荆州城区 凤凰涅槃——荆州城 荆州长江大桥 省属重点大学——长江大学 荆州花鼓戏 词条图册 更多图册 解读词条背后的知识 无风起念 文史研